In [110]:
from pyscf import gto, scf, fci
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumCircuit

from scipy.optimize import minimize
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import TwoLocal
from qiskit_ionq import IonQProvider
from qiskit.quantum_info import Statevector
import os
from qiskit.quantum_info import Pauli
from qiskit_algorithms import NumPyEigensolver
from qiskit.circuit.library import HGate , SdgGate
from qiskit import transpile


In [111]:
def fermion_to_qubit(problem, second_q_op, mapper_name):
  if mapper_name == "JW":
    mapper = JordanWignerMapper()
  if mapper_name == "Pa":
    mapper = ParityMapper(num_particles=problem.num_particles)
  if mapper_name == "BK":
    mapper = BravyiKitaevMapper()
  qubit_op = mapper.map(second_q_op)
  return qubit_op , mapper
# 고전값 비교용 FCI 
# H2 인경우로만 해뒀고, 거리 주면 에너지 계산할 수 있음. 

def cost_func(parameter,H_matrix, n_ptl, ansatz, backend):
    params = ansatz.parameters 
    param_dict = dict(zip(params, parameter))
    qc = ansatz.assign_parameters(param_dict)
    qc.measure_all()
    job = backend.run(qc, shots=2000)
    result = job.get_probabilities()
    all_basis = []
    all_prob = []
    for bitstring, prob in result.items():
        count_ones = bitstring.count('1')
        half = len(bitstring) // 2
        left_ones = bitstring[:half].count('1')
        right_ones = bitstring[half:].count('1')
        if count_ones == n_ptl and left_ones==right_ones :
            all_basis.append(bitstring)
            all_prob.append(prob)

    all_basis = np.array(all_basis)
    all_prob = np.array(all_prob)
    top_indices = np.argsort(-all_prob)[:20]
    probable_basis = all_basis[top_indices]
    print("most probable_basis : ", probable_basis)

    n = len(probable_basis)
    H = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            basis_i = probable_basis[i]
            basis_j = probable_basis[j]
            sv1 = Statevector.from_label(basis_i)
            sv2 = Statevector.from_label(basis_j)
            inner_product = sv1.conjugate().data @ (H_matrix @ sv2.data)
            H_ij = np.real(inner_product)
            H[i, j] = H_ij 
            #print("<{}|H|{}> = {}".format(basis_i,basis_j,H_ij))
    eigvals, eigvecs = np.linalg.eig(H)
    Energy = np.min(eigvals)

    return Energy

In [112]:
api_key = "7IMO85iuzmzAAgri4osG5BPTeCFXrKoM"
provider = IonQProvider(api_key)
simulator_backend = provider.get_backend("simulator")
#simulator_backend = provider.get_backend("qpu.forte-1")

In [101]:
# O at origin
O = (0.000, 0.000, 0.000)

# O–H bond length
bond_length = 0.958  # in angstroms
angle_deg = 104.5
angle_rad = np.deg2rad(angle_deg / 2)

# 좌우 대칭 구조로 H 원자 배치
H1 = (bond_length * np.sin(angle_rad), bond_length * np.cos(angle_rad), 0.0)
H2 = (-bond_length * np.sin(angle_rad), bond_length * np.cos(angle_rad), 0.0)

# 최종 입력
atoms = ["O", "H", "H"]
coords = [O, H1, H2]
basis = "sto3g"
charge = 0
multiplicity = 1

Co_O_moleculeinfo = MoleculeInfo(atoms, coords, charge=charge, multiplicity=multiplicity)
driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
E_problem = driver.run() # 여기는 이후, As_transformer 로 변경. 

fermionic_hamiltonian = E_problem.hamiltonian
second_q_op = fermionic_hamiltonian.second_q_op()
repulsion = fermionic_hamiltonian.constants['nuclear_repulsion_energy']
hamiltonian, mapper = fermion_to_qubit(E_problem, second_q_op, "JW")
H_op = hamiltonian.to_operator()
H_matrix = H_op.data
num_qubits = hamiltonian.num_qubits
num_particles = E_problem.num_particles
num_spatial_orbitals = E_problem.num_spatial_orbitals




init_state = HartreeFock(num_spatial_orbitals,num_particles,mapper)
ansatz = TwoLocal(num_spatial_orbitals*2, ['ry', 'rz'], 'cz', initial_state=init_state).decompose()
#ansatz = UCCSD(num_spatial_orbitals,num_particles,mapper,initial_state=init_state)

pauli_basis = hamiltonian.paulis
coeffs = hamiltonian.coeffs
num_params = ansatz.num_parameters
x0 = 2 * np.pi * np.random.random(num_params)

params = ansatz.parameters 
param_dict = dict(zip(params, x0))
Energy = 0
qc = ansatz.assign_parameters(param_dict)
qc.measure_all()
job = simulator_backend.run(qc, shots=20000)
result = job.get_probabilities()
print(result)

{'00000000000000': 2.4000000848900527e-05, '00000000000001': 0.00015100000018719584, '00000000000101': 0.00019500000053085387, '00000000000111': 0.0001289999927394092, '00000000001001': 1.4999999621068127e-05, '00000000001100': 1.700000029813964e-05, '00000000001101': 7.999999797903001e-05, '00000000010010': 1.4000000192027073e-05, '00000000010101': 0.0004239999980200082, '00000000011110': 4.999999873689376e-06, '00000000011111': 5.900000178371556e-05, '00000000100001': 1.9999999494757503e-05, '00000000100010': 9.999999747378752e-06, '00000000100100': 5.500000042957254e-05, '00000000100101': 0.00048600000445730984, '00000000100110': 3.400000059627928e-05, '00000000100111': 1.2000000424450263e-05, '00000000101001': 0.00010299999848939478, '00000000101100': 1.700000029813964e-05, '00000000101111': 7.999999797903001e-05, '00000000110000': 4.5000000682193786e-05, '00000000110001': 0.0002610000083222985, '00000000110010': 3.300000025774352e-05, '00000000110011': 0.00014200000441633165, '000

In [105]:
all_basis = []
all_prob = []
n_ptl = np.sum(num_particles)
np.sum
for bitstring, prob in result.items():
    count_ones = bitstring.count('1')
    half = len(bitstring) // 2
    left_ones = bitstring[:half].count('1')
    right_ones = bitstring[half:].count('1')
    if count_ones == n_ptl and left_ones==right_ones :
        all_basis.append(bitstring)
        all_prob.append(prob)

all_basis = np.array(all_basis)
all_prob = np.array(all_prob)
top_indices = np.argsort(-all_prob)[:20]
probable_basis = all_basis[top_indices]
print("most probable_basis : ", probable_basis)

n = len(probable_basis)
H = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        basis_i = probable_basis[i]
        basis_j = probable_basis[j]
        sv1 = Statevector.from_label(basis_i)
        sv2 = Statevector.from_label(basis_j)
        inner_product = sv1.conjugate().data @ (H_matrix @ sv2.data)
        H_ij = np.real(inner_product)
        H[i, j] = H_ij 
        #print("<{}|H|{}> = {}".format(basis_i,basis_j,H_ij))
print(H)
eigvals, eigvecs = np.linalg.eig(H)
print(eigvals + repulsion)

most probable_basis :  ['11111001110011' '01111101111010' '11111001110110' '11111000111011'
 '11111001111010' '11101101111010' '11111001100111' '11111001101011'
 '10111101110011' '10111100101111' '11111000111110' '11101100111110'
 '11111000110111' '10111100111110' '11111001110101' '11111001111001'
 '01111100111110' '11111001101110' '10111100110111' '01111101111001']
<11111001110011|H|11111001110011> = -60.929243368317444
<11111001110011|H|01111101111010> = 0.0
<11111001110011|H|11111001110110> = 0.0
<11111001110011|H|11111000111011> = 0.0
<11111001110011|H|11111001111010> = 0.14531624684458827
<11111001110011|H|11101101111010> = 0.018923693870685643
<11111001110011|H|11111001100111> = 0.0
<11111001110011|H|11111001101011> = 0.0
<11111001110011|H|10111101110011> = 0.19168508848299387
<11111001110011|H|10111100101111> = 0.0
<11111001110011|H|11111000111110> = 0.008808477250641772
<11111001110011|H|11101100111110> = 0.0
<11111001110011|H|11111000110111> = -0.4248678148271612
<111110011100

In [108]:
print(np.min(eigvals + repulsion))

-72.59864838926762
